<a href="https://colab.research.google.com/github/kush9405/Implementation/blob/main/video_features.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# prompt: set the path correctly of opencv-contrib-python as /usr/local/lib/python3.11/dist-packages

# You might need to adjust the path based on your specific environment
import sys
sys.path.append('/usr/local/lib/python3.11/dist-packages')

# Now you can import cv2
import cv2
cv2.__version__


In [ ]:
import cv2
import os
import time
import numpy as np
import concurrent.futures

def preprocess_frame(frame, use_cuda=True):
    """
    Preprocesses a video frame with background subtraction, resizing, and grayscale conversion.

    Args:
        frame (numpy.ndarray): Input frame (BGR format).
        use_cuda (bool): Whether to use CUDA for processing.
    Returns:
        numpy.ndarray: Preprocessed frame (grayscale).
    """
    try:
        # Convert to grayscale immediately (faster processing, less data)
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) #Remove colors
        small = cv2.resize(gray, (320, 240)) #Small Images
        # Background Subtraction (adaptive Gaussian mixture model)
        fgbg = cv2.createBackgroundSubtractorMOG2()
        fgmask = fgbg.apply(small) # Apply back ground.

        # Further Enhance (noise removal, etc. if needed)

        # Convert to NumPy array
        fgmask = np.array(fgmask)
        return fgmask

    except Exception as e:
        print(f"Error preprocessing frame: {e}")
        return None

def extract_video_features(video_path, use_cuda=True):
    """
    Extracts motion history images (MHI) from a video file, which is faster and allows background subtraction to be used.
    Also implements CUDA options

    Args:
        video_path (str): Path to the input video file.
        use_cuda (bool): Whether to use CUDA for processing.

    Returns:
        tuple: Contains the first frame for reference, and the motion history images.
    """
    try:
        # Open video
        video = cv2.VideoCapture(video_path)
        if not video.isOpened():
            print(f"Could not open video: {video_path}")
            return None

        ret, first_frame = video.read()
        # Get video properties
        fps = video.get(cv2.CAP_PROP_FPS)
        width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
        # If it does not work, return the first frame

        #Define a kernel Size to blur, and create the MHI
        motion_history = np.zeros((height, width), np.float32)
        duration = 0.5 #Threshold for the value of MHI
        # Process frames
        num_frames = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
        for i in range(num_frames): #For each of the frames, do the work
            ret, frame = video.read()
            if not ret:
                break #Break and stop operation

            preprocessed_frame = preprocess_frame(frame)

            #Update Motion History.  Blur, update motion images, and rescale
            if hasattr(cv2, 'updateMotionHistory'):
                timestamp = i / fps #Timestamp updates depending on the time frame
                cv2.blur(preprocessed_frame, (3,3), preprocessed_frame) #blur is the first step to remove noise
                cv2.updateMotionHistory(preprocessed_frame, motion_history, timestamp, duration) #This is where MHI happens
                cv2.normalize(motion_history, None, 0, 255, cv2.NORM_MINMAX) #Normalize
            else:
                 print("Skipping cv2.updateMotionHistory as it's not available") #Alert it broke


        video.release()

        return first_frame, motion_history #Return the preprocessed first frame

    except Exception as e:
        print(f"Error extracting video features: {e}")
        return None, None

def process_video_file(video_file, output_folder, use_cuda):
    """Processes the video file, extracts the motion history image, and save results."""
    print(f"Processing video: {video_file}")

    file_name = os.path.basename(video_file)
    file_name_without_ext, file_ext = os.path.splitext(file_name)
    output_mhi_image_path = os.path.join(output_folder, f"{file_name_without_ext}_mhi.jpg")
    output_firstframe_image_path = os.path.join(output_folder, f"{file_name_without_ext}_firstframe.jpg")
    try:
        first_frame, motion_history = extract_video_features(video_file, use_cuda) #Get the results

        if first_frame is not None and motion_history is not None:
            cv2.imwrite(output_mhi_image_path, motion_history) #Save the Motion Histories
            cv2.imwrite(output_firstframe_image_path, first_frame) #Save the first frame for easy visual reference
        else:
            print("No features extracted, or failure happened.") #Alert it broke
    except Exception as e:
        print(f"Error processing: {e}") #Print exception

def traverse_directory(root_directory, output_root_folder, max_workers=4, use_cuda=False):
    """Traverses a directory and processes all video files in parallel."""
    video_files = []
    for foldername, subfolders, filenames in os.walk(root_directory):
        for filename in filenames:
            if filename.lower().endswith(('.mp4')):
                input_file_path = os.path.join(foldername, filename)
                relative_path = os.path.relpath(foldername, root_directory) #Relative Path for consistncy
                output_folder = os.path.join(output_root_folder, relative_path) #To a specific directory
                os.makedirs(output_folder, exist_ok=True)
                video_files.append((input_file_path, output_folder)) #Add the results into a file

    with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
        executor.map(lambda item: process_video_file(item[0], item[1], use_cuda), video_files)

if __name__ == '__main__':
    # Make sure these are valid directories to load in
    input_directory = "/content/drive/MyDrive/Datasets/preprocess1" # Load video from here
    output_directory = "/content/drive/MyDrive/VideoFeatures/" # Output feature to this directory
    traverse_directory(input_directory, output_directory, max_workers=4, use_cuda=True) # Use CUDA and traverse

    print("Code Runs Correctly")

In [ ]:
# --- 1. Uninstall Existing OpenCV (and ensure the notebook is clean) ---
!pip uninstall opencv-python -y
!pip uninstall opencv-contrib-python -y


Found existing installation: opencv-contrib-python 4.11.0.86
Uninstalling opencv-contrib-python-4.11.0.86:
  Successfully uninstalled opencv-contrib-python-4.11.0.86


In [ ]:
# --- Download and Check out ---
!git clone https://github.com/opencv/opencv.git
!git clone https://github.com/opencv/opencv_contrib.git

Cloning into 'opencv'...
remote: Enumerating objects: 343174, done.
remote: Counting objects: 100% (195/195), done.
remote: Compressing objects: 100% (142/142), done.
remote: Total 343174 (delta 120), reused 53 (delta 53), pack-reused 342979 (from 3)
Receiving objects: 100% (343174/343174), 532.05 MiB | 28.88 MiB/s, done.
Resolving deltas: 100% (239453/239453), done.
Updating files: 100% (7609/7609), done.
fatal: destination path 'opencv_contrib' already exists and is not an empty directory.


In [ ]:
OPENCV_VERSION = "4.8.0"  # Replace with your desired version
%cd opencv
!git checkout $OPENCV_VERSION
%cd ../opencv_contrib
!git checkout $OPENCV_VERSION
%cd ..

# --- Create Build Directory ---
!mkdir build
%cd build

/content/opencv
Note: switching to '4.8.0'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by switching back to a branch.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -c with the switch command. Example:

  git switch -c <new-branch-name>

Or undo this operation with:

  git switch -

Turn off this advice by setting config variable advice.detachedHead to false

HEAD is now at f9a59f2592 Release OpenCV 4.8.0
/content/opencv_contrib
Note: switching to '4.8.0'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by switching back to a branch.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -c with the switch c

In [ ]:
!pip install scikit-build

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.6/85.6 kB 3.8 MB/s eta 0:00:00


In [ ]:
!scikit-build --help

/bin/bash: line 1: scikit-build: command not found


In [ ]:
!cmake -D CMAKE_BUILD_TYPE=Release \
      -D CMAKE_INSTALL_PREFIX=/usr/local \
-D WITH_TBB=ON  \
-D BUILD_opencv_python3=ON  \
-D OPENCV_GENERATE_PKGCONFIG=ON \
-D BUILD_EXAMPLES=OFF \
-D OPENCV_EXTRA_MODULES_PATH=../opencv_contrib/modules \
../opencv

CMake Deprecation Warning at CMakeLists.txt:27 (cmake_minimum_required):
  Compatibility with CMake < 3.10 will be removed from a future version of
  CMake.

  Update the VERSION argument <min> value.  Or, use the <min>...<max> syntax
  to tell CMake that the project requires at least <min> but has been updated
  to work with policies introduced by <max> or earlier.


CMake Warning (dev) at CMakeLists.txt:113 (enable_language):
  project() should be called prior to this enable_language() call.
This warning is for project developers.  Use -Wno-dev to suppress it.

-- Detected processor: x86_64
CMake Warning (dev) at cmake/OpenCVUtils.cmake:144 (find_package):
  Policy CMP0148 is not set: The FindPythonInterp and FindPythonLibs modules
  are removed.  Run "cmake --help-policy CMP0148" for policy details.  Use
  the cmake_policy command to set the policy and suppress this warning.

Call Stack (most recent call first):
  cmake/OpenCVDetectPython.cmake:64 (find_host_package)
  cmake/OpenCVD

In [ ]:
!cmake \
    -D CMAKE_BUILD_TYPE=Release \
    -D CMAKE_INSTALL_PREFIX=/usr/local \
    -D WITH_CUDA=ON \
    -D CUDA_ARCH_BIN="7.5" \
    -D CUDA_ARCH_PTX="" \
    -D WITH_CUDNN=ON \
    -D OPENCV_DNN_CUDA=ON \
    -D ENABLE_FAST_MATH=1 \
    -D CUDA_FAST_MATH=1 \
    -D WITH_TBB=ON \
    -D BUILD_opencv_python3=ON \
    -D OPENCV_GENERATE_PKGCONFIG=ON \
    -D BUILD_EXAMPLES=OFF \
    -D OPENCV_EXTRA_MODULES_PATH=/usr/local/lib/python3.11/dist-packages \
    -D CUDA_TOOLKIT_ROOT_DIR=/usr/local/cuda \
    ../opencv

CMake Deprecation Warning at CMakeLists.txt:27 (cmake_minimum_required):
  Compatibility with CMake < 3.10 will be removed from a future version of
  CMake.

  Update the VERSION argument <min> value.  Or, use the <min>...<max> syntax
  to tell CMake that the project requires at least <min> but has been updated
  to work with policies introduced by <max> or earlier.


CMake Warning (dev) at CMakeLists.txt:113 (enable_language):
  project() should be called prior to this enable_language() call.
This warning is for project developers.  Use -Wno-dev to suppress it.

-- Detected processor: x86_64
CMake Warning (dev) at cmake/OpenCVUtils.cmake:144 (find_package):
  Policy CMP0148 is not set: The FindPythonInterp and FindPythonLibs modules
  are removed.  Run "cmake --help-policy CMP0148" for policy details.  Use
  the cmake_policy command to set the policy and suppress this warning.

Call Stack (most recent call first):
  cmake/OpenCVDetectPython.cmake:64 (find_host_package)
  cmake/OpenCVD

In [ ]:
!find nanobind-config.cmake

find: ‘nanobind-config.cmake’: No such file or directory


In [ ]:
!cmake -D CMAKE_BUILD_TYPE=Release \
       -D CMAKE_INSTALL_PREFIX=/usr/local \
       -D WITH_CUDA=ON \
       -D CUDA_ARCH_BIN="7.5" \
       -D CUDA_ARCH_PTX="" \
       -D WITH_CUDNN=ON \
       -D OPENCV_DNN_CUDA=ON \
       -D ENABLE_FAST_MATH=1 \
       -D CUDA_FAST_MATH=1 \
       -D WITH_TBB=ON \
       -D BUILD_opencv_python3=ON \
       -D OPENCV_GENERATE_PKGCONFIG=ON \
       -D BUILD_EXAMPLES=OFF \
       -D OPENCV_EXTRA_MODULES_PATH=../opencv_contrib/modules \
       -D CUDA_TOOLKIT_ROOT_DIR=/usr/local/cuda \
       -D nanobind_DIR=/usr/local/lib/python3.11/site-packages \  # Specify the directory.  It is required.
       ../opencv

In [ ]:
!make -j$(nproc)
!sudo make install
!sudo ldconfig

[  0%] Built target opencv_highgui_plugins
[  0%] Built target opencv_dnn_plugins
[  0%] Built target opencv_videoio_plugins
[  0%] Generate opencv4.pc
[  1%] Building C object 3rdparty/ippiw/CMakeFiles/ippiw.dir/src/iw_core.c.o
CMake Deprecation Warning at /content/opencv/cmake/OpenCVGenPkgconfig.cmake:113 (cmake_minimum_required):
  Compatibility with CMake < 3.10 will be removed from a future version of
  CMake.

  Update the VERSION argument <min> value.  Or, use the <min>...<max> syntax
  to tell CMake that the project requires at least <min> but has been updated
  to work with policies introduced by <max> or earlier.


[  1%] Built target gen-pkgconfig
[  1%] Building C object 3rdparty/ittnotify/CMakeFiles/ittnotify.dir/src/ittnotify/ittnotify_static.c.o
[  1%] Building C object 3rdparty/ippiw/CMakeFiles/ippiw.dir/src/iw_image.c.o
[  1%] Building C object 3rdparty/ittnotify/CMakeFiles/ittnotify.dir/src/ittnotify/jitprofiling.c.o
[  1%] Building C object 3rdparty/ippiw/CMakeFiles/

In [ ]:
import sys
sys.path.append('/usr/local/lib/python3.10/site-packages')# Adjust if needed
print(sys.path)

['/content', '/env/python', '/usr/lib/python311.zip', '/usr/lib/python3.11', '/usr/lib/python3.11/lib-dynload', '', '/usr/local/lib/python3.11/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.11/dist-packages/IPython/extensions', '/root/.ipython', '/usr/local/lib/python3.10/site-packages', '/usr/local/lib/python3.10/site-packages', '/usr/local/lib/python3.10/site-packages']


In [ ]:
!pip install opencv-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.0/63.0 MB 12.8 MB/s eta 0:00:00


In [ ]:
import cv2
print(cv2.__version__)  # If this works, you're on the right track

4.11.0


In [ ]:
# prompt: delete all the files from a directory

import os
import shutil

def delete_files_in_directory(directory_path):
  """Deletes all files within a specified directory."""
  for filename in os.listdir(directory_path):
    file_path = os.path.join(directory_path, filename)
    try:
      if os.path.isfile(file_path) or os.path.islink(file_path):
        os.unlink(file_path)
      elif os.path.isdir(file_path):
        shutil.rmtree(file_path)
    except Exception as e:
      print('Failed to delete %s. Reason: %s' % (file_path, e))

# Example usage:
directory_to_clear = "/content/VideoFeatures"  # Replace with your directory
delete_files_in_directory(directory_to_clear)


In [ ]:
import cv2
import os
import time
import numpy as np
import concurrent.futures

def preprocess_frame(frame, use_cuda=True):
    """
    Preprocesses a video frame with background subtraction, resizing, and grayscale conversion.

    Args:
        frame (numpy.ndarray): Input frame (BGR format).
        use_cuda (bool): Whether to use CUDA for processing (not directly applicable here).

    Returns:
        numpy.ndarray: Preprocessed frame (grayscale).
    """
    try:
        # Convert to grayscale immediately (faster processing, less data)
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) #Remove colors
        small = cv2.resize(gray, (320, 240)) #Small Images
        # Background Subtraction (adaptive Gaussian mixture model)
        fgbg = cv2.createBackgroundSubtractorMOG2()
        fgmask = fgbg.apply(small) # Apply background subtraction

        # Convert to NumPy array
        fgmask = np.array(fgmask, dtype=np.uint8)  # Explicit data type for consistency
        return fgmask

    except Exception as e:
        print(f"Error preprocessing frame: {e}")
        return None

def extract_video_features(video_path, use_cuda=True):
    """
    Extracts Histogram of Oriented Gradients (HOG) features from a video frame,
    aiming for faster and more stable features.

    Args:
        video_path (str): Path to the input video file.
        use_cuda (bool): Whether to use CUDA for processing (not directly applicable for HOG here).

    Returns:
        tuple: Contains the first frame for reference, and the HOG descriptors as a NumPy array.
    """
    try:
        # Open video
        video = cv2.VideoCapture(video_path)
        if not video.isOpened():
            print(f"Could not open video: {video_path}")
            return None, None

        ret, first_frame = video.read()
        if not ret:
            print("Could not read the first frame")
            return None, None
        if first_frame is not None:
            preprocessed_frame = preprocess_frame(first_frame)

            # Initialize HOG descriptor
            hog = cv2.HOGDescriptor() #No Params is fine, set defaults in here!

            # Calculate HOG descriptors
            hog_descriptors = hog.compute(preprocessed_frame) #Calculate HOG Description
        else:
            hog_descriptors = None #Cannot complete operation, if there is no HOG descrip

        video.release()

        return first_frame, hog_descriptors

    except Exception as e:
        print(f"Error extracting video features: {e}")
        return None, None

def process_video_file(video_file, output_folder, use_cuda):
    """Processes the video file, extracts the HOG descriptor, and saves results as NumPy array."""
    print(f"Processing video: {video_file}")

    file_name = os.path.basename(video_file)
    file_name_without_ext, file_ext = os.path.splitext(file_name)
    output_hog_features_path = os.path.join(output_folder, f"{file_name_without_ext}_hog.npy")  # Change extension to .npy
    output_firstframe_image_path = os.path.join(output_folder, f"{file_name_without_ext}_firstframe.jpg")

    try:
        first_frame, hog_descriptors = extract_video_features(video_file, use_cuda) #Get the results

        if first_frame is not None and hog_descriptors is not None:
            cv2.imwrite(output_firstframe_image_path, first_frame) #Save the first frame for easy visual reference

            # Save HOG descriptors as a NumPy array
            np.save(output_hog_features_path, hog_descriptors)  # Changed from savetxt
            print(f"HOG features saved to {output_hog_features_path}")
        else:
            print("No features extracted, or failure happened.") #Alert it broke
    except Exception as e:
        print(f"Error processing: {e}") #Print exception

def traverse_directory(root_directory, output_root_folder, max_workers=4, use_cuda=False):
    """Traverses a directory and processes all video files in parallel."""
    video_files = []
    for foldername, subfolders, filenames in os.walk(root_directory):
        for filename in filenames:
            if filename.lower().endswith(('.mp4')):
                input_file_path = os.path.join(foldername, filename)
                relative_path = os.path.relpath(foldername, root_directory) #Relative Path for consistncy
                output_folder = os.path.join(output_root_folder, relative_path) #To a specific directory
                os.makedirs(output_folder, exist_ok=True)
                video_files.append((input_file_path, output_folder)) #Add the results into a file

    with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
        executor.map(lambda item: process_video_file(item[0], item[1], use_cuda), video_files)

if __name__ == '__main__':
    # Make sure these are valid directories to load in
    input_directory = "/content/drive/MyDrive/Datasets/preprocess1" # Load video from here
    output_directory = "/content/drive/MyDrive/VideoFeatures" # Output feature to this directory
    traverse_directory(input_directory, output_directory, max_workers=2, use_cuda=True) # Use CUDA and traverse

    print("Code Runs Correctly")

Streaming output truncated to the last 5000 lines.
Processing video: /content/drive/MyDrive/Datasets/dataset/1005-2004/v=5Tnl7_8RqlA__#00-16-10_00-17-19_label_B6-0-0.mp4
HOG features saved to /content/drive/MyDrive/VideoFeatures/./Love.Actually.2003__#01-47-05_01-54-40_label_A_video_hog.npy
Processing video: /content/drive/MyDrive/Datasets/preprocess1/Love.Actually.2003__#01-55-05_01-59-40_label_A_video.mp4
Error processing: [Errno 2] No such file or directory: '/content/drive/MyDrive/VideoFeatures/1005-2004/v=5Tnl7_8RqlA__#00-13-00_00-14-09_label_B6-0-0_hog.npy'
Processing video: /content/drive/MyDrive/Datasets/dataset/1005-2004/v=5Tnl7_8RqlA__#00-08-35_00-10-25_label_B6-0-0.mp4
Error processing: [Errno 2] No such file or directory: '/content/drive/MyDrive/VideoFeatures/1005-2004/v=5Tnl7_8RqlA__#00-14-10_00-16-09_label_B6-0-0_hog.npy'
Processing video: /content/drive/MyDrive/Datasets/dataset/1005-2004/v=5TPecJ1aUs8__#1_label_A.mp4
HOG features saved to /content/VideoFeatures/./Deadpoo